In [ ]:
# Required imports
from langchain.document_loaders import UnstructuredMarkdownLoader, NotebookLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from typing import List, Dict
from datetime import datetime
import glob
from dotenv import load_dotenv, find_dotenv
import os



# Load environment variables
load_dotenv(find_dotenv())

# Custom prompt template
custom_prompt_template = """You are a helpful AI assistant specialized in demand forecasting and related topics.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}

Question: {question}

Please provide your answer following these guidelines:
1. Use ONLY information from the provided context
2. ALWAYS cite your sources using [Source X] format where X is the source number
3. If multiple sources support a statement, cite all relevant sources: [Source 1,2]
4. If the context doesn't contain enough information, clearly state that
5. Structure your response in a clear, logical manner
6. Keep the answer focused and relevant to demand forecasting

Remember: Every significant statement should have a source citation.

Answer: Let me help you with that.
"""

# Create the prompt template
PROMPT = PromptTemplate(
    template=custom_prompt_template, input_variables=["context", "question"]
)


def load_markdown_files(directory: str = "./data/docs/") -> List:
    """Load markdown files from directory"""
    loader = DirectoryLoader(
        directory, glob="**/*.md", loader_cls=UnstructuredMarkdownLoader
    )
    return loader.load()


def load_jupyter_notebooks(directory: str = "./demand_forecast_notebooks/") -> List:
    """Load jupyter notebooks from directory"""
    documents = []
    for notebook_path in glob.glob(f"{directory}/**/*.ipynb", recursive=True):
        if ".ipynb_checkpoints" not in notebook_path:
            try:
                loader = NotebookLoader(
                    notebook_path,
                    include_outputs=True,
                    max_output_length=50,
                    remove_newline=True,
                )
                documents.extend(loader.load())
            except Exception as e:
                print(f"Error loading notebook {notebook_path}: {e}")
    return documents


def process_documents(documents: List, chunk_size: int = 1000, chunk_overlap: int = 20):
    """Split documents into chunks"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n## ", "\n### ", "\n#### ", "\n", " ", ""],
    )
    return text_splitter.split_documents(documents)


def create_vector_store(documents: List, persist_directory: str = "./data/chroma_db"):
    """Create and persist vector store"""
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    # Remove the persist() call as it's now automatic
    return vectorstore

def setup_qa_chain(vectorstore):
    """Setup the QA chain with custom prompt"""
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature=0,
    )

    # Create the QA chain with custom prompt
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
        chain_type_kwargs={"prompt": PROMPT, "verbose": True},
        return_source_documents=True,
    )

    return qa_chain


def extract_relevant_context(doc) -> Dict:
    """Extract and format relevant context from a document."""
    return {
        "content": doc.page_content,
        "source": doc.metadata.get("source", "Unnamed Source"),
        "page": doc.metadata.get("page", None),
        "chunk": doc.metadata.get("chunk", None),
    }


def format_citation(source_info: Dict) -> str:
    """Format source information into a citation."""
    citation = source_info["source"]
    if source_info["page"]:
        citation += f", page {source_info['page']}"
    return citation


def ask_question(qa_chain, question: str) -> str:
    """Ask a question and return a well-formatted answer with citations."""
    try:
        # Get the answer and source documents
        result = qa_chain({"query": question})

        # Extract answer and sources
        answer = result["result"]
        sources = result.get("source_documents", [])

        # Create source mapping
        source_map = {}
        for idx, doc in enumerate(sources, 1):
            source_info = extract_relevant_context(doc)
            source_map[idx] = source_info

        # Format the response in Markdown
        md = f"### Question\n{question}\n\n"
        md += f"### Answer\n{answer}\n\n"

        if source_map:
            md += "### References\n\n"
            for idx, source_info in source_map.items():
                citation = format_citation(source_info)
                md += f"[Source {idx}] {citation}\n"
                excerpt = source_info["content"][:200].replace("\n", " ").strip()
                md += f"> {excerpt}...\n\n"

        return md
    except Exception as e:
        return f"**An error occurred:** {str(e)}"


def interactive_qa(qa_chain):
    """Interactive Q&A session with formatted Markdown output and citations."""
    try:
        from IPython.display import display, Markdown, HTML

        use_markdown = True
    except ImportError:
        use_markdown = False

    session_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    header = f"""
    # Demand Forecasting Q&A Session
    Session started: {session_time}

    Enter your questions below. Type 'exit' to end the session.
    """

    if use_markdown:
        display(Markdown(header))
    else:
        print(header)

    while True:
        question = input("\nYour question: ").strip()
        if question.lower() == "exit":
            footer = "\n### Session Ended\nThank you for using the Q&A system!"
            if use_markdown:
                display(Markdown(footer))
            else:
                print(footer)
            break

        if use_markdown:
            display(Markdown("---\n*Processing your question...*"))
        else:
            print("\nProcessing your question...")

        answer_md = ask_question(qa_chain, question)

        if use_markdown:
            display(Markdown(answer_md))
            display(Markdown("---"))
        else:
            print(answer_md)
            print("---")


def main():
    print("Loading documents...")
    markdown_docs = load_markdown_files("../data/docs/")
    notebook_docs = load_jupyter_notebooks("./demand_forecast_notebooks/")

    all_documents = markdown_docs + notebook_docs
    print(
        f"Loaded {len(markdown_docs)} markdown files and {len(notebook_docs)} notebooks"
    )

    print("Processing documents...")
    splits = process_documents(all_documents)
    print(f"Created {len(splits)} splits")

    print("Creating vector store...")
    vectorstore = create_vector_store(splits)
    print("Vector store created and persisted")

    qa_chain = setup_qa_chain(vectorstore)
    return qa_chain


if __name__ == "__main__":
    qa_chain = main()
    interactive_qa(qa_chain)

# Update it to use LCEL
## remove markdown output
## chat history added
## warnings are updated

In [ ]:
# Required imports
from langchain.document_loaders import UnstructuredMarkdownLoader, NotebookLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from typing import List, Dict
from datetime import datetime
import glob
from dotenv import load_dotenv, find_dotenv
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from typing import List, Dict, Tuple

# Load environment variables
load_dotenv(find_dotenv())

# Custom prompt template
custom_prompt_template = """You are a helpful AI assistant specialized in demand forecasting and related topics.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}

Question: {question}

Please provide your answer following these guidelines:
1. Use ONLY information from the provided context
2. ALWAYS cite your sources using [Source X] format where X is the source number
3. If multiple sources support a statement, cite all relevant sources: [Source 1,2]
4. If the context doesn't contain enough information, clearly state that
5. Structure your response in a clear, logical manner
6. Keep the answer focused and relevant to demand forecasting

Remember: Every significant statement should have a source citation.

Answer: Let me help you with that.
"""

# Create the prompt template
PROMPT = PromptTemplate(
    template=custom_prompt_template,
    input_variables=["context", "question"]
)

def load_markdown_files(directory: str = "./data/docs/") -> List:
    """Load markdown files from directory"""
    loader = DirectoryLoader(
        directory,
        glob="**/*.md",
        loader_cls=UnstructuredMarkdownLoader
    )
    return loader.load()

def load_jupyter_notebooks(directory: str = "./demand_forecast_notebooks/") -> List:
    """Load jupyter notebooks from directory"""
    documents = []
    for notebook_path in glob.glob(f"{directory}/**/*.ipynb", recursive=True):
        if ".ipynb_checkpoints" not in notebook_path:
            try:
                loader = NotebookLoader(
                    notebook_path,
                    include_outputs=True,
                    max_output_length=50,
                    remove_newline=True
                )
                documents.extend(loader.load())
            except Exception as e:
                print(f"Error loading notebook {notebook_path}: {e}")
    return documents

def process_documents(documents: List, chunk_size: int = 1000, chunk_overlap: int = 20):
    """Split documents into chunks"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n## ", "\n### ", "\n#### ", "\n", " ", ""]
    )
    return text_splitter.split_documents(documents)

def create_vector_store(documents: List, persist_directory: str = "../data/chroma_db"):
    """Create and persist vector store"""
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    return vectorstore

def extract_relevant_context(doc) -> Dict:
    """Extract and format relevant context from a document."""
    return {
        "content": doc.page_content,
        "source": doc.metadata.get("source", "Unnamed Source"),
        "page": doc.metadata.get("page", None),
    }

def format_citation(source_info: Dict) -> str:
    """Format source information into a citation."""
    citation = source_info["source"]
    if source_info["page"]:
        citation += f", page {source_info['page']}"
    return citation



In [4]:
class ConversationalRetriever:
    def __init__(self, vectorstore):
        self.vectorstore = vectorstore
        self.memory = []  # List to store conversation history

    def get_relevant_documents(self, query: str) -> Tuple[List, str]:
        # Updated to use invoke instead of get_relevant_documents
        retriever = self.vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={
                "k": 3,
                "lambda_mult": 0.7
            }
        )
        # Use invoke instead of get_relevant_documents
        docs = retriever.invoke(query)
        return docs

    def add_message(self, question: str, answer: str):
        """Add a message pair to memory"""
        self.memory.append(HumanMessage(content=question))
        self.memory.append(AIMessage(content=answer))

    def get_chat_history(self) -> str:
        """Format chat history into a string"""
        if not self.memory:
            return ""

        return "\n".join(
            f"Human: {msg.content}" if isinstance(msg, HumanMessage)
            else f"Assistant: {msg.content}"
            for msg in self.memory
        )

def setup_qa_chain(vectorstore):
    """Setup the QA chain using LCEL with conversation memory"""
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature=0,
    )

    # Initialize the conversational retriever
    conv_retriever = ConversationalRetriever(vectorstore)

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Updated prompt template with chat history
    conv_prompt = PromptTemplate(
        template="""You are a helpful AI assistant specialized in demand forecasting and related topics.

Previous conversation:
{chat_history}

Use the following context to answer the question. If you reference something from the previous conversation,
make it explicit by saying "As discussed earlier..." or similar phrases.

Context: {context}

Current Question: {question}

Please provide your answer following these guidelines:
1. Use ONLY information from the provided context and previous conversation
2. ALWAYS cite your sources using [Source X] format where X is the source number
3. If multiple sources support a statement, cite all relevant sources: [Source 1,2]
4. If the context doesn't contain enough information, clearly state that
5. Structure your response in a clear, logical manner
6. Keep the answer focused and relevant to demand forecasting

Answer: Let me help you with that.
""",
        input_variables=["chat_history", "context", "question"]
    )

    def get_response(input_dict):
        # Get relevant documents using invoke
        docs = conv_retriever.get_relevant_documents(input_dict["question"])

        # Prepare the input for the chain
        chain_input = {
            "context": format_docs(docs),
            "question": input_dict["question"],
            "chat_history": conv_retriever.get_chat_history()
        }

        # Generate response
        response = conv_prompt.format(**chain_input)
        response = llm.invoke(response)
        response = StrOutputParser().invoke(response)

        # Add to conversation history
        conv_retriever.add_message(input_dict["question"], response)

        return {"result": response, "source_documents": docs}

    final_chain = RunnablePassthrough() | get_response

    return final_chain

def ask_question(qa_chain, question: str) -> str:
    """Ask a question and return a formatted answer with citations."""
    try:
        result = qa_chain.invoke({"question": question})

        answer = result["result"]
        sources = result.get("source_documents", [])

        # Create a dictionary to track unique sources
        unique_sources = {}
        for doc in sources:
            source_info = extract_relevant_context(doc)
            # Create a unique key based on source and content
            key = (source_info["source"], source_info["content"][:100])
            if key not in unique_sources:
                unique_sources[key] = source_info

        # Format the response as plain text
        response = f"Question: {question}\n\n"
        response += f"Answer: {answer}\n\n"

        if unique_sources:
            response += "References:\n"
            for idx, (_, source_info) in enumerate(unique_sources.items(), 1):
                citation = format_citation(source_info)
                response += f"[Source {idx}] {citation}\n"
                excerpt = source_info["content"][:150].replace("\n", " ").strip()
                response += f"Excerpt: {excerpt}...\n\n"

        return response
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
def interactive_qa(qa_chain):
    """Interactive Q&A session with chat memory and plain text output."""
    print("\nDemand Forecasting Q&A Session")
    print(f"Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("\nEnter your questions below. Type 'exit' to end the session.")
    print("Type 'history' to see the conversation history.")
    print("-" * 50)

    # Initialize conversation history
    conversation_history = []

    while True:
        question = input("\nYour question: ").strip()

        # Handle special commands
        if question.lower() == "exit":
            print("\nSession Ended. Thank you for using the Q&A system!")
            break

        if question.lower() == "history":
            print("\nConversation History:")
            if not conversation_history:
                print("No previous conversations.")
            else:
                for i, (q, a) in enumerate(conversation_history, 1):
                    print(f"\nDialog {i}:")
                    print(f"Q: {q}")
                    print(f"A: {a}")
            print("-" * 50)
            continue

        print("\nProcessing your question...")

        # Add conversation context to the question
        if conversation_history:
            context = "\n".join([
                f"Q: {q}\nA: {a}"
                for q, a in conversation_history[-3:]  # Last 3 conversations
            ])
            print("\nUsing context from previous conversations...")

        # Get the answer
        answer_text = ask_question(qa_chain, question)

        # Extract just the answer part (without the references) for history
        answer_only = answer_text.split("References:")[0].strip()

        # Store in conversation history
        conversation_history.append((question, answer_only))

        # Print the full response including references
        print("\n" + answer_text)
        print("-" * 50)

        # If the conversation is getting too long, keep only the last N interactions
        if len(conversation_history) > 5:  # Keep last 5 conversations
            conversation_history = conversation_history[-5:]

def main():
    print("Loading documents...")
    markdown_docs = load_markdown_files("../data/docs/")
    notebook_docs = load_jupyter_notebooks("./demand_forecast_notebooks/")

    all_documents = markdown_docs + notebook_docs
    print(f"Loaded {len(markdown_docs)} markdown files and {len(notebook_docs)} notebooks")

    print("Processing documents...")
    splits = process_documents(all_documents)
    print(f"Created {len(splits)} splits")

    print("Creating vector store...")
    vectorstore = create_vector_store(splits)
    print("Vector store created and persisted")

    # Create the chain with memory
    qa_chain = setup_qa_chain(vectorstore)
    return qa_chain

In [ ]:
if __name__ == "__main__":
    qa_chain = main()
    interactive_qa(qa_chain)

# the memory still not perfoming perfect. it needs more attention. 

Loading documents...
Loaded 5 markdown files and 13 notebooks
Processing documents...
Created 485 splits
Creating vector store...
Vector store created and persisted

Demand Forecasting Q&A Session
Session started: 2025-04-30 22:34:03

Enter your questions below. Type 'exit' to end the session.
Type 'history' to see the conversation history.
--------------------------------------------------

Processing your question...

Question: is there a mentino to arima here

Answer: Yes, there is a mention of ARIMA in the provided context. It states that a process $X_t$ is ARIMA(p,d,q) if and only if $\nabla^d X_t$ is a stationary ARMA(p,q) [Source 1]. This highlights the relationship between ARIMA and ARMA models, where differencing the series at appropriate lags allows for the extension from ARMA to ARIMA to address issues such as trend and seasonality in time series forecasting.

References:
[Source 1] demand_forecast_notebooks/ts-2-linear-vision.ipynb
Excerpt: 'markdown' cell: '<a id="section-